In [35]:
from collections import Counter
from itertools import combinations
import pandas as pd
import numpy as np
import re
import string
from tqdm import tqdm_notebook
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score, GroupKFold
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.linear_model import LinearRegression, Ridge, Lasso

tqdm_notebook.pandas()

pd.set_option('display.max_columns' , 999)
pd.options.display.max_colwidth = 300

import fastText

In [2]:
df_train = pd.read_csv('data/train.tsv', sep='\t', quotechar=' ', quoting=3, header=None, encoding='utf-8')
df_train.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply', 'label', 'confidence']
# df_test = pd.read_csv('data/public.tsv', sep='\t', quotechar=' ', quoting=3, header=None, encoding='utf-8')
df_test = pd.read_csv('data/final.tsv', sep='\t', quotechar=' ', quoting=3, header=None, encoding='utf-8')
df_test.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply']

In [3]:
print(df_train.shape)
df_train.head(n=7)

(97533, 8)


,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.875352
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.900968
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.884320
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.982530
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.838054
5,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",5,просто повесь трубку .,bad,0.955718
6,50117448291,бывало и получше .,"слушайте , мы с женой . . . совсем заблудились .","я пытаюсь добраться до юджина , но , кажется ,...",0,едете ?,bad,0.909115


In [4]:
print(df_test.shape)
df_test.head(n=7)

(104834, 6)


,context_id,context_2,context_1,context_0,reply_id,reply
0,4909294510,NaN,NaN,нет . . . у тебя на лице написано - нет .,0,тогда я попытался с двумя другими женщинами и ...
1,4909294510,NaN,NaN,нет . . . у тебя на лице написано - нет .,1,"я улыбаюсь потому . . . потому что , описывая ..."
2,4909294510,NaN,NaN,нет . . . у тебя на лице написано - нет .,2,"это , так сказать , соответствует уровню моей ..."
3,4909294510,NaN,NaN,нет . . . у тебя на лице написано - нет .,3,я врач .
4,4909294510,NaN,NaN,нет . . . у тебя на лице написано - нет .,4,не обращайте на меня внимания !
5,4909294510,NaN,NaN,нет . . . у тебя на лице написано - нет .,5,извините .
6,15805100619,NaN,сейчас непросто думать .,весь напичкан лекарствами .,0,успокоит желудок .


In [5]:
df_train.fillna('', inplace=True)
df_test.fillna('', inplace=True)

# FastText

In [6]:
ft_model = fastText.load_model('./data/cc.ru.300.bin')

In [7]:
def fasttext_feature(df):
    return (df['context_2'] + ' ' + df['context_1'] + ' ' + df['context_0'] + ' ' + df['reply']).apply(
        lambda r: ft_model.get_sentence_vector(r)
    )

column_names = ['ft_vec_{}'.format(i) for i in range(300)]
X_train = pd.DataFrame(np.vstack(fasttext_feature(df_train)), columns=column_names)
X_test  = pd.DataFrame(np.vstack(fasttext_feature(df_test)),  columns=column_names)

In [8]:
X_train.head()

,ft_vec_0,ft_vec_1,ft_vec_2,ft_vec_3,ft_vec_4,ft_vec_5,ft_vec_6,ft_vec_7,ft_vec_8,ft_vec_9,ft_vec_10,ft_vec_11,ft_vec_12,ft_vec_13,ft_vec_14,ft_vec_15,ft_vec_16,ft_vec_17,ft_vec_18,ft_vec_19,ft_vec_20,ft_vec_21,ft_vec_22,ft_vec_23,ft_vec_24,ft_vec_25,ft_vec_26,ft_vec_27,ft_vec_28,ft_vec_29,ft_vec_30,ft_vec_31,ft_vec_32,ft_vec_33,ft_vec_34,ft_vec_35,ft_vec_36,ft_vec_37,ft_vec_38,ft_vec_39,ft_vec_40,ft_vec_41,ft_vec_42,ft_vec_43,ft_vec_44,ft_vec_45,ft_vec_46,ft_vec_47,ft_vec_48,ft_vec_49,ft_vec_50,ft_vec_51,ft_vec_52,ft_vec_53,ft_vec_54,ft_vec_55,ft_vec_56,ft_vec_57,ft_vec_58,ft_vec_59,ft_vec_60,ft_vec_61,ft_vec_62,ft_vec_63,ft_vec_64,ft_vec_65,ft_vec_66,ft_vec_67,ft_vec_68,ft_vec_69,ft_vec_70,ft_vec_71,ft_vec_72,ft_vec_73,ft_vec_74,ft_vec_75,ft_vec_76,ft_vec_77,ft_vec_78,ft_vec_79,ft_vec_80,ft_vec_81,ft_vec_82,ft_vec_83,ft_vec_84,ft_vec_85,ft_vec_86,ft_vec_87,ft_vec_88,ft_vec_89,ft_vec_90,ft_vec_91,ft_vec_92,ft_vec_93,ft_vec_94,ft_vec_95,ft_vec_96,ft_vec_97,ft_vec_98,ft_vec_99,ft_vec_100,ft_vec_101,ft_vec_102,ft_vec_103,ft_vec_104,ft_vec_105,ft_vec_106,ft_vec_107,ft_vec_108,ft_vec_109,ft_vec_110,ft_vec_111,ft_vec_112,ft_vec_113,ft_vec_114,ft_vec_115,ft_vec_116,ft_vec_117,ft_vec_118,ft_vec_119,ft_vec_120,ft_vec_121,ft_vec_122,ft_vec_123,ft_vec_124,ft_vec_125,ft_vec_126,ft_vec_127,ft_vec_128,ft_vec_129,ft_vec_130,ft_vec_131,ft_vec_132,ft_vec_133,ft_vec_134,ft_vec_135,ft_vec_136,ft_vec_137,ft_vec_138,ft_vec_139,ft_vec_140,ft_vec_141,ft_vec_142,ft_vec_143,ft_vec_144,ft_vec_145,ft_vec_146,ft_vec_147,ft_vec_148,ft_vec_149,ft_vec_150,ft_vec_151,ft_vec_152,ft_vec_153,ft_vec_154,ft_vec_155,ft_vec_156,ft_vec_157,ft_vec_158,ft_vec_159,ft_vec_160,ft_vec_161,ft_vec_162,ft_vec_163,ft_vec_164,ft_vec_165,ft_vec_166,ft_vec_167,ft_vec_168,ft_vec_169,ft_vec_170,ft_vec_171,ft_vec_172,ft_vec_173,ft_vec_174,ft_vec_175,ft_vec_176,ft_vec_177,ft_vec_178,ft_vec_179,ft_vec_180,ft_vec_181,ft_vec_182,ft_vec_183,ft_vec_184,ft_vec_185,ft_vec_186,ft_vec_187,ft_vec_188,ft_vec_189,ft_vec_190,ft_vec_191,ft_vec_192,ft_vec_193,ft_vec_194,ft_vec_195,ft_vec_196,ft_vec_197,ft_vec_198,ft_vec_199,ft_vec_200,ft_vec_201,ft_vec_202,ft_vec_203,ft_vec_204,ft_vec_205,ft_vec_206,ft_vec_207,ft_vec_208,ft_vec_209,ft_vec_210,ft_vec_211,ft_vec_212,ft_vec_213,ft_vec_214,ft_vec_215,ft_vec_216,ft_vec_217,ft_vec_218,ft_vec_219,ft_vec_220,ft_vec_221,ft_vec_222,ft_vec_223,ft_vec_224,ft_vec_225,ft_vec_226,ft_vec_227,ft_vec_228,ft_vec_229,ft_vec_230,ft_vec_231,ft_vec_232,ft_vec_233,ft_vec_234,ft_vec_235,ft_vec_236,ft_vec_237,ft_vec_238,ft_vec_239,ft_vec_240,ft_vec_241,ft_vec_242,ft_vec_243,ft_vec_244,ft_vec_245,ft_vec_246,ft_vec_247,ft_vec_248,ft_vec_249,ft_vec_250,ft_vec_251,ft_vec_252,ft_vec_253,ft_vec_254,ft_vec_255,ft_vec_256,ft_vec_257,ft_vec_258,ft_vec_259,ft_vec_260,ft_vec_261,ft_vec_262,ft_vec_263,ft_vec_264,ft_vec_265,ft_vec_266,ft_vec_267,ft_vec_268,ft_vec_269,ft_vec_270,ft_vec_271,ft_vec_272,ft_vec_273,ft_vec_274,ft_vec_275,ft_vec_276,ft_vec_277,ft_vec_278,ft_vec_279,ft_vec_280,ft_vec_281,ft_vec_282,ft_vec_283,ft_vec_284,ft_vec_285,ft_vec_286,ft_vec_287,ft_vec_288,ft_vec_289,ft_vec_290,ft_vec_291,ft_vec_292,ft_vec_293,ft_vec_294,ft_vec_295,ft_vec_296,ft_vec_297,ft_vec_298,ft_vec_299
0,0.017006,-0.034744,0.006785,-0.001589,-0.000538,0.004771,-0.016154,0.004372,-0.026210,-0.019262,0.002816,-0.003943,0.007769,0.003796,-0.017581,-0.020314,-0.013450,0.003917,-0.026513,0.002963,-0.005929,-0.001393,0.001359,0.026505,-0.013441,-0.008334,0.008402,0.033696,0.006745,0.003382,-0.015411,-0.004139,-0.005801,0.026962,-0.004942,-0.015525,0.003415,-0.306534,-0.023072,-0.005365,-0.016269,-0.008223,-0.004202,-0.021127,-0.021455,0.015516,-0.001456,0.016348,-0.011353,-0.005339,-0.000371,-0.013523,0.000366,-0.001340,-0.011223,-0.011227,-0.000396,0.002062,-0.042276,0.012103,-0.130811,-0.012966,0.020520,-0.008974,-0.005352,-0.007379,-0.003888,-0.002084,0.000176,-0.007150,-0.010111,0.010237,0.020073,-0.022171,-0.005727,0.006169,-0.014142,-0.021247,0.004208,0.009417,0.005247,0.020148,0.005188,-0.015143

# Simple Features

In [9]:
def preprocess_feature(X):
    '''
        X : dataframe
            Dataset for preprocess
    '''
    X = X.copy()
    X['context_2_notnull'] = X['context_2'].notnull().astype('int8')
    X['context_1_notnull'] = X['context_1'].notnull().astype('int8')

    X.fillna('', inplace=True)
    X['is_duplicate_reply'] = X['reply'].duplicated(keep=False).astype('int8')

    return X

In [10]:
def vowels_count(x):
    '''
        x : string
            string for get vowels count
    '''
    return len(re.findall(vowels, x))


def consonant_count(x):
    '''
        x : string
            string for get consonant count
    '''
    return len(re.findall(consonant, x))


def divide_vov_by_cons(x):
    '''
        x : string
            string for get ratio of vowels to consonants
    '''
    return vowels_count(x)/(consonant_count(x) + 0.001)


def count_word(x):
    '''
        x : string
            string for get count of word in string
    '''
    return len(x.split(' '))

func = [len, vowels_count, consonant_count, divide_vov_by_cons, count_word]
func_name = ['len', 'vowels_count', 'consonant_count','divide_vov_by_cons', 'count_word']

In [11]:
def drop_punctuation(x):
    '''
        x : string
            String from which the punctuation is removed
    '''
    return re.sub('[%s]' % string.punctuation, ' ', x)

def simple_feature(X):
    '''
        X : dataframe
            Dataset for get first feature
    '''
    for col in ['reply', 'context_2', 'context_1', 'context_0']:
        X[col] = X[col].apply(drop_punctuation)

    for col in ['context_2', 'context_1', 'context_0', 'reply']:
        for f, name in zip(func, func_name):
            X[col + '_' + name] = X[col].apply(f)

    return X

In [12]:
vowels = '[аеиоуыэюя]'
consonant = '[бвгджзйклмнпрстфхцчшщъь]'
alphabet = vowels[:-1] + consonant[1:]

X_train1 = simple_feature(preprocess_feature(df_train))
X_test1  = simple_feature(preprocess_feature(df_test))

In [13]:
X_test1.head()

,context_id,context_2,context_1,context_0,reply_id,reply,context_2_notnull,context_1_notnull,is_duplicate_reply,context_2_len,context_2_vowels_count,context_2_consonant_count,context_2_divide_vov_by_cons,context_2_count_word,context_1_len,context_1_vowels_count,context_1_consonant_count,context_1_divide_vov_by_cons,context_1_count_word,context_0_len,context_0_vowels_count,context_0_consonant_count,context_0_divide_vov_by_cons,context_0_count_word,reply_len,reply_vowels_count,reply_consonant_count,reply_divide_vov_by_cons,reply_count_word
0,4909294510,,,нет у тебя на лице написано нет,0,тогда я попытался с двумя другими женщинами и ...,1,1,1,0,0,0,0.0,1,0,0,0,0.0,1,41,12,13,0.923006,17,71,28,30,0.933302,14
1,4909294510,,,нет у тебя на лице написано нет,1,я улыбаюсь потому потому что описывая ...,1,1,0,0,0,0,0.0,1,0,0,0,0.0,1,41,12,13,0.923006,17,93,34,34,0.999971,26
2,4909294510,,,нет у тебя на лице написано нет,2,это так сказать соответствует уровню моей ...,1,1,0,0,0,0,0.0,1,0,0,0,0.0,1,41,12,13,0.923006,17,56,18,26,0.692281,13
3,4909294510,,,нет у тебя на лице написано нет,3,я врач,1,1,0,0,0,0,0.0,1,0,0,0,0.0,1,41,12,13,0.923006,17,8,2,3,0.666445,4
4,4909294510,,,нет у тебя на лице написано нет,4,не обращайте на меня внимания,1,1,0,0,0,0,0.0,1,0,0,0,0.0,1,41,12,13,0.923006,17,31,12,13,0.923006,7


In [14]:
print(set(X_train1.columns[10:]) ^ set(X_test1.columns[6:]))
feature_columns = list(X_train1.columns[10:])

{'context_1_notnull', 'context_2_notnull'}


In [15]:
X_train2 = pd.concat([
    X_train1[feature_columns],
    X_train
], axis=1)

X_test2  = pd.concat([
    X_test1[feature_columns],
    X_test
], axis=1)

# TF-IDF

In [39]:
df_train['context_all'] = \
    df_train['context_2'] + ' ' + \
    df_train['context_1'] + ' ' + \
    df_train['context_0'] + ' ' + \
    df_train['reply']

df_test['context_all'] = \
    df_test['context_2'] + ' ' + \
    df_test['context_1'] + ' ' + \
    df_test['context_0'] + ' ' + \
    df_test['reply']

content = df_train['context_all'].append(df_test['context_all'])
content.head()

0                   кликни на меня а потом на надпись " видео - звонок " . о , я тебя вижу . ладно , повесь трубку . не могу .
1      кликни на меня а потом на надпись " видео - звонок " . о , я тебя вижу . ладно , повесь трубку . нет , звонить буду я .
2    кликни на меня а потом на надпись " видео - звонок " . о , я тебя вижу . ладно , повесь трубку . слушай , я не мог уйти .
3     кликни на меня а потом на надпись " видео - звонок " . о , я тебя вижу . ладно , повесь трубку . я не прекращу звонить .
4                кликни на меня а потом на надпись " видео - звонок " . о , я тебя вижу . ладно , повесь трубку . я звоню им .
Name: context_all, dtype: object

In [40]:
model_tfidf = TfidfVectorizer().fit(content)

In [ ]:
model_tfidf

# Target

In [16]:
map_label = {'bad': 0, 'neutral': 1, 'good': 2}

df_train['label_map'] = df_train['label'].map(map_label)

df_train['target'] = 0
df_train.loc[df_train['label_map'] == 0, 'target'] = 1 - df_train.loc[df_train['label_map'] == 0, 'confidence']
df_train.loc[df_train['label_map'] == 1, 'target'] = df_train.loc[df_train['label_map'] == 1, 'confidence']
df_train.loc[df_train['label_map'] == 2, 'target'] = 2 * df_train.loc[df_train['label_map'] == 2, 'confidence']

y_train = df_train['target']

# Train and Predict

## XGBoost - 85192.00

In [17]:
model = XGBRegressor(objective='rank:pairwise' , max_depth=5)
model.fit(X_train2, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='rank:pairwise', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [18]:
df_test['target'] = model.predict(X_test2)

In [19]:
submission = \
    df_test.sort_values(by=['context_id', 'target'], ascending=False)[['context_id', 'reply_id']]
    
submission.to_csv('myltsev_submission__xgboost.tsv', sep='\t', header=False , index=False)

## LogisticRegression

In [20]:
model_ridge = Ridge()
model_ridge.fit(X_train2, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [21]:
submission = pd.DataFrame()

submission['context_id'] = df_test['context_id']
submission['reply_id'] = df_test['reply_id']
submission['rank'] = -model.predict(X_test2)

submission.sort_values(by=['context_id', 'rank'], inplace=True)

In [22]:
submission.head()

,context_id,reply_id,rank
1,4909294510,1,-0.641196
2,4909294510,2,-0.592023
4,4909294510,4,-0.521804
0,4909294510,0,-0.430454
5,4909294510,5,-0.123040


In [23]:
submission.to_csv('myltsev_submission__xgboost__logreg_ridge.tsv',header=None, index=False, sep=' ', columns=['context_id', 'reply_id'])

In [ ]:
def fetch_sentences(column_name: str):
    return df_train[~pd.isnull(df_train[column_name])][column_name].values

sentences = set(fetch_sentences('context_2')) | \
            set(fetch_sentences('context_1')) | \
            set(fetch_sentences('context_0'))
sentences

In [ ]:
with open('data/sentences.txt', 'w') as f:
    for sentence in sentences:
        f.write(sentence + '\n')

In [ ]:
def context2vec(df, context_name):
    print('handling', context_name)
    with open('data/sentences_temp.txt', 'w') as f:
        for sent in df[context_name].values:
            f.write(str(sent) + '\n')
    ! ../fastText/build/fasttext print-sentence-vectors data/cc.ru.300.bin < data/sentences_temp.txt > data/sentences_temp_vec.txt
    df[context_name + '_vec'] = \
        pd.read_csv('data/sentences_temp_vec.txt', header=None)[0] \
            .apply(lambda s: np.fromstring(s, dtype=float, sep=' '))
    ! rm data/sentences_temp.txt data/sentences_temp_vec.txt
    df[context_name + '_vec'] = \
        np.where(df[context_name].isnull(), df[context_name], df[context_name + '_vec'])

context2vec(df_train, 'context_2')
context2vec(df_train, 'context_1')
context2vec(df_train, 'context_0')
context2vec(df_train, 'reply')

context2vec(df_test, 'context_2')
context2vec(df_test, 'context_1')
context2vec(df_test, 'context_0')
context2vec(df_test, 'reply')

In [ ]:
df_train.sample(n=5)

In [ ]:
df_test.sample(n=5)

In [ ]:
def label2num(st):
    if st == 'good':
        return 3
    elif st == 'neutral':
        return 2
    else:
        return 1

df_train['label_num'] = df_train['label'].apply(label2num)

In [ ]:
X_train = df_train[['context_2_vec', 'context_1_vec', 'context_0_vec', 'reply_vec']]
X_train.sample(n=5)

In [ ]:
y_train = df_train['confidence'] * df_train['label_num']
y_train.sample(n=5)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

reg = Ridge()
reg.fit(X_train, y_train)